In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/My Drive/my_code/XLNet-and-NER

/content/drive/My Drive/my_code/XLNet-and-NER


In [3]:
ls

config.json  dev.txt  pytorch_model.bin  train.txt  XLNet-and-NER.ipynb


In [ ]:
# !pip install torch torchvision
# !pip install pytorch_transformers

In [5]:
train_samples = []
train_labels = []

In [ ]:
with open('./train.txt', 'r') as f:
    while True:
        s1 = f.readline()
        if not s1:
            # 如果读取到内容为空，则读取结束
            break
        s2 = f.readline()
        _ = f.readline()
        train_samples.append(s1.replace('\n', ''))
        train_labels.append(s2.replace('\n', ''))

In [7]:
len(train_samples), len(train_labels)

(10279, 10279)

In [8]:
from pytorch_transformers import XLNetTokenizer
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

100%|██████████| 798011/798011 [00:00<00:00, 868982.72B/s]


In [9]:
input_ids = []
input_labels = []

In [10]:
for text, ori_labels in zip(train_samples, train_labels):
    l = text.split(' ')
    labels = []
    text_ids = []
    for word, label in zip(l, ori_labels):
        if word == '':
            continue
        tokens = tokenizer.tokenize(word)
        for i, j in enumerate(tokens):
            if i == 0:
                labels.append(int(label))
                text_ids.append(tokenizer.convert_tokens_to_ids(j))
            else:
                labels.append(9)
                text_ids.append(tokenizer.convert_tokens_to_ids(j))
    input_ids.append(text_ids)
    input_labels.append(labels)

In [11]:
input_ids[:3]

[[1534, 8006, 23, 871, 547, 22, 8569, 559, 15174, 17, 9],
 [1656, 19769],
 [322, 24405, 4342, 10642, 2025, 13, 3383, 13, 2896]]

In [12]:
input_labels[:3]

[[1, 0, 9, 7, 0, 0, 0, 7, 0, 0, 9], [3, 4], [5, 9, 9, 9, 0, 9, 9, 9, 9]]

In [13]:
len(input_ids), len(input_labels)

(10279, 10279)

In [14]:
import torch
from torch.utils.data import DataLoader, TensorDataset

del train_samples
del train_labels

for j in range(len(input_ids)):
    # 将样本数据填充至长度为 128
    i = input_ids[j]
    if len(i) != 128:
        input_ids[j].extend([0]*(128 - len(i)))

for j in range(len(input_labels)):
    # 将样本数据填充至长度为 128
    i = input_labels[j]
    if len(i) != 128:
        input_labels[j].extend([10]*(128 - len(i)))

# 构建数据集和数据迭代器，设定 batch_size 大小为 8
train_set = TensorDataset(torch.LongTensor(input_ids),
                          torch.LongTensor(input_labels))
train_loader = DataLoader(dataset=train_set,
                          batch_size=8,
                          shuffle=True)
train_loader

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [16]:
import torch.nn as nn
from pytorch_transformers import XLNetModel

class NERModel(nn.Module):
    def __init__(self, num_class=11):
        super(NERModel, self).__init__()
        # 读取 XLNet 预训练模型
        self.model = XLNetModel.from_pretrained("./")
        self.dropout = nn.Dropout(0.1)
        self.l1 = nn.Linear(768, num_class)

    def forward(self, x, attention_mask=None):
        outputs = self.model(x, attention_mask=attention_mask)
        x = outputs[0]  # 形状为 batch * seqlen * 768
        x = self.dropout(x)
        x = self.l1(x)
        return x

In [17]:
def loss_function(logits, target, masks, num_class=11):
    criterion = nn.CrossEntropyLoss(reduction='none')
    logits = logits.view(-1, num_class)
    target = target.view(-1)
    masks = masks.view(-1)
    cross_entropy = criterion(logits, target)
    loss = cross_entropy * masks
    loss = loss.sum() / (masks.sum() + 1e-12)  # 加上 1e-12 防止被除数为 0
    loss = loss.to(device)
    return loss

In [18]:
from torch.optim import Adam

model = NERModel()
model.to(device)
model.train()

optimizer = Adam(model.parameters(), lr=1e-5)

In [19]:
from torch.autograd import Variable
import time

pre = time.time()

epoch = 3

for i in range(epoch):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).to(device), Variable(target).to(device)

        optimizer.zero_grad()

        # 生成掩膜
        mask = []
        for sample in data:
            mask.append([1 if i != 0 else 0 for i in sample])
        mask = torch.FloatTensor(mask).to(device)

        output = model(data, attention_mask=mask)

        # 得到模型预测结果
        pred = torch.argmax(output, dim=2)

        loss = loss_function(output, target, mask)
        loss.backward()

        optimizer.step()

        if ((batch_idx+1) % 10) == 1:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss:{:.6f}'.format(
                i+1, batch_idx, len(train_loader), 100. *
                batch_idx/len(train_loader), loss.item()
            ))

        if batch_idx == len(train_loader)-1:
            # 在每个 Epoch 的最后输出一下结果
            print('labels:', target)
            print('pred:', pred)

print('训练时间：', time.time()-pre)

Train Epoch: 1 [0/1285 (0%)]	Loss:5.729533
Train Epoch: 1 [10/1285 (1%)]	Loss:1.644814
Train Epoch: 1 [20/1285 (2%)]	Loss:1.354358
Train Epoch: 1 [30/1285 (2%)]	Loss:1.466767
Train Epoch: 1 [40/1285 (3%)]	Loss:0.979642
Train Epoch: 1 [50/1285 (4%)]	Loss:1.046154
Train Epoch: 1 [60/1285 (5%)]	Loss:1.242516
Train Epoch: 1 [70/1285 (5%)]	Loss:0.987815
Train Epoch: 1 [80/1285 (6%)]	Loss:1.170622
Train Epoch: 1 [90/1285 (7%)]	Loss:0.902470
Train Epoch: 1 [100/1285 (8%)]	Loss:1.048250
Train Epoch: 1 [110/1285 (9%)]	Loss:0.762536
Train Epoch: 1 [120/1285 (9%)]	Loss:0.875750
Train Epoch: 1 [130/1285 (10%)]	Loss:0.792331
Train Epoch: 1 [140/1285 (11%)]	Loss:0.799931
Train Epoch: 1 [150/1285 (12%)]	Loss:0.524697
Train Epoch: 1 [160/1285 (12%)]	Loss:0.426843
Train Epoch: 1 [170/1285 (13%)]	Loss:0.471425
Train Epoch: 1 [180/1285 (14%)]	Loss:0.534094
Train Epoch: 1 [190/1285 (15%)]	Loss:0.369024
Train Epoch: 1 [200/1285 (16%)]	Loss:0.230637
Train Epoch: 1 [210/1285 (16%)]	Loss:0.405183
Train Epoch:

In [20]:
!nvidia-smi

Sun Sep 13 03:30:31 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    42W / 250W |   4075MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [21]:
eval_samples = []
eval_labels = []

with open('./dev.txt', 'r') as f:
    while True:
        s1 = f.readline()
        if not s1:
            break
        s2 = f.readline()
        _ = f.readline()
        eval_samples.append(s1.replace('\n', ''))
        eval_labels.append(s2.replace('\n', ''))

len(eval_samples)

2184

In [22]:
# 这里使用和训练集同样的方式修改标签，不再赘述
input_ids = []
input_labels = []
for text, ori_labels in zip(eval_samples, eval_labels):
    l = text.split(' ')
    labels = []
    text_ids = []
    for word, label in zip(l, ori_labels):
        if word == '':
            continue
        tokens = tokenizer.tokenize(word)
        for i, j in enumerate(tokens):
            if i == 0:
                labels.append(int(label))
                text_ids.append(tokenizer.convert_tokens_to_ids(j))
            else:
                labels.append(9)
                text_ids.append(tokenizer.convert_tokens_to_ids(j))
    input_ids.append(text_ids)
    input_labels.append(labels)

del eval_samples
del eval_labels

for j in range(len(input_ids)):
    # 将样本数据填充至长度为 128
    i = input_ids[j]
    if len(i) != 128:
        input_ids[j].extend([0]*(128 - len(i)))

for j in range(len(input_labels)):
    # 将样本数据填充至长度为 128
    i = input_labels[j]
    if len(i) != 128:
        input_labels[j].extend([10]*(128 - len(i)))

# 构建数据集和数据迭代器，设定 batch_size 大小为 1
eval_set = TensorDataset(torch.LongTensor(input_ids),
                         torch.LongTensor(input_labels))
eval_loader = DataLoader(dataset=eval_set,
                         batch_size=1,
                         shuffle=False)
eval_loader

In [23]:
from tqdm.notebook import tqdm

model.eval()

correct = 0
total = 0

for batch_idx, (data, target) in enumerate(tqdm(eval_loader)):
    data = data.to(device)
    target = target.float().to(device)

    # 生成掩膜
    mask = []
    for sample in data:
        mask.append([1 if i != 0 else 0 for i in sample])
    mask = torch.Tensor(mask).to(device)

    output = model(data, attention_mask=mask)

    # 得到模型预测结果
    pred = torch.argmax(output, dim=2)

    # 将掩膜添加到预测结果上，便于计算准确率
    pred = pred.float()
    pred = pred * mask
    target = target * mask

    pred = pred[:, 0:mask.sum().int().item()]
    target = target[:, 0:mask.sum().int().item()]

    correct += (pred == target).sum().item()
    total += mask.sum().item()

print('正确分类的标签数：{}，标签总数：{}，准确率：{:.2f}%'.format(
    correct, total, 100.*correct/total))


正确分类的标签数：46478，标签总数：46836.0，准确率：99.24%
